In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# utilities.prompts

> Fill in a module description here

In [9]:
#| default_exp utilities.prompts

In [10]:
#| hide
from nbdev.showdoc import *

In [11]:
#| export
from typing import ClassVar

from langchain.prompts import PromptTemplate, BasePromptTemplate
from pydantic import BaseModel, Field

from langGraph_swarm.utilities.prompts_components import *
# from crewai.utilities import I18N

In [12]:
#| export
class Prompts(BaseModel):
    """Manages and generates prompts for a generic agent with support for different languages."""

    i18n: I18N = Field(default=I18N())

    SCRATCHPAD_SLICE: ClassVar[str] = "\n{agent_scratchpad}"

    def task_execution_with_memory(self) -> BasePromptTemplate:
        """Generate a prompt for task execution with memory components."""
        return self._build_prompt(["role_playing", "tools", "memory", "task"])

    def task_execution_without_tools(self) -> BasePromptTemplate:
        """Generate a prompt for task execution without tools components."""
        return self._build_prompt(["role_playing", "task"])

    def task_execution(self) -> BasePromptTemplate:
        """Generate a standard prompt for task execution."""
        return self._build_prompt(["role_playing", "tools", "task"])
    
    def task_execution_openai(self) -> BasePromptTemplate:
        """Generate a standard prompt for task execution."""
        return self._build_prompt(["role_playing", "tools", "task"], openai=True)

    def _build_prompt(self, components: list[str], openai: bool = False) -> BasePromptTemplate:
        """Constructs a prompt string from specified components."""
        prompt_parts = [self.i18n.slice(component) for component in components] if not openai else [self.i18n.openai_fn(component) for component in components]
        prompt_parts.append(self.SCRATCHPAD_SLICE)
        return PromptTemplate.from_template("".join(prompt_parts))

In [14]:
Prompts().task_execution_openai()

PromptTemplate(input_variables=['agent_scratchpad', 'backstory', 'goal', 'input', 'role', 'tools'], template='You are {role}.\n{backstory}\n\nYour personal goal is: {goal}TOOLS:\n------\nYou have access to only the following tools:\n\n{tools}\n\nBegin! This is VERY important to you, your job depends on it!\n\nCurrent Task: {input}\n{agent_scratchpad}')

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()